In [ ]:
! pip install parsivar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 36.2 MB 337 kB/s 
     |████████████████████████████████| 1.5 MB 35.4 MB/s 
  Created wheel for parsivar: filename=parsivar-0.2.3-py3-none-any.whl size=36492972 sha256=077970d4eae9462617092dd5ecc3e5457820fe14981baf1018304b8a31358b86
  Stored in directory: /root/.cache/pip/wheels/ae/67/7a/49cbf08f64d3f76a26eceaf0e481a40e233f05d4356875cbed
  Created wheel for nltk: filename=nltk-3.4.5-py3-none-any.whl size=1449923 sha256=848bd73a8be70b774fc3df1cba41fa600bb36bdcba39f86b04bf07cc1069dff2
  Stored in directory: /root/.cache/pip/wheels/48/8b/7f/473521e0c731c6566d631b281f323842bbda9bd819eb9a3ead
Successfully built parsivar nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
! gdown 1Bo_WQLGQD5UrPv3IJdV7cdVZtSHeJSQB
! unzip data.zip

Downloading...
From: https://drive.google.com/uc?id=1Bo_WQLGQD5UrPv3IJdV7cdVZtSHeJSQB
To: /content/data.zip
100% 2.45M/2.45M [00:00<00:00, 193MB/s]
Archive:  data.zip
   creating: data/
  inflating: data/test.en            
  inflating: data/test.fa            
  inflating: data/train.en           
  inflating: data/train.fa           
  inflating: data/valid.en           
  inflating: data/valid.fa           


## Preprocess

In [ ]:
from parsivar import Normalizer
from tqdm import tqdm
!mkdir preprocessed_data
my_normalizer = Normalizer(pinglish_conversion_needed = True,
                           statistical_space_correction = True)
files = ['test.en','test.fa','train.en','train.fa','valid.en','valid.fa']
for file_name in tqdm(files):
  with open('data/'+file_name) as f:
    lines = f.readlines()
    lines = [line.rstrip() for line in lines]
  for line in lines:
    if(file_name.endswith(".fa")):
      text = my_normalizer.normalize(line)
    else:
      text = line.lower()
    with open('preprocessed_data/'+file_name, 'a') as f:
      f.write(text+'\n')

100%|██████████| 6/6 [00:32<00:00,  5.36s/it]


## OpenNMT

### BPE

In [ ]:
!git clone -b legacy https://github.com/OpenNMT/OpenNMT-py

Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 17675, done.
remote: Total 17675 (delta 0), reused 0 (delta 0), pack-reused 17675
Receiving objects: 100% (17675/17675), 273.61 MiB | 30.94 MiB/s, done.
Resolving deltas: 100% (12760/12760), done.


In [ ]:
!pip install -r ./OpenNMT-py/floyd_requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pytorch/text to /tmp/pip-req-build-786graun
  Running command git clone -q https://github.com/pytorch/text /tmp/pip-req-build-786graun
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  ERROR: Failed building wheel for torchtext
Failed to build torchtext
ERROR: Could not build wheels for torchtext which use PEP 517 and cannot be installed directly


In [ ]:
! mkdir BPE

In [ ]:
!python OpenNMT-py/tools/learn_bpe.py -i preprocessed_data/train.en -o BPE/src.code -s 10000

In [ ]:
!python OpenNMT-py/tools/learn_bpe.py -i preprocessed_data/train.fa -o BPE/tgt.code -s 10000

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c BPE/src.code -i preprocessed_data/train.en -o BPE/train-bpe.en

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c BPE/src.code -i preprocessed_data/valid.en -o BPE/valid-bpe.en

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c BPE/src.code -i preprocessed_data/test.en -o BPE/test-bpe.en

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c BPE/tgt.code -i preprocessed_data/train.fa -o BPE/train-bpe.fa

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c BPE/tgt.code -i preprocessed_data/valid.fa -o BPE/valid-bpe.fa

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c BPE/tgt.code -i preprocessed_data/test.fa -o BPE/test-bpe.fa

In [ ]:
!nvidia-smi

Fri Jun 10 09:44:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### openNMT preprocess

In [ ]:
!pip install torchtext==0.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.5 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0


In [ ]:
!pip install configargparse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir translate

In [ ]:
!python OpenNMT-py/preprocess.py -train_src BPE/train-bpe.en -train_tgt BPE/train-bpe.fa -valid_src BPE/valid-bpe.en -valid_tgt BPE/valid-bpe.fa -save_data translate/data -overwrite

[2022-06-10 09:44:26,554 INFO] Extracting features...
[2022-06-10 09:44:26,555 INFO]  * number of source features: 0.
[2022-06-10 09:44:26,555 INFO]  * number of target features: 0.
[2022-06-10 09:44:26,555 INFO] Building `Fields` object...
[2022-06-10 09:44:26,555 INFO] Building & saving training data...
[2022-06-10 09:44:26,620 INFO] Building shard 0.
[2022-06-10 09:44:27,696 INFO]  * saving 0th train data shard to translate/data.train.0.pt.
[2022-06-10 09:44:29,606 INFO]  * tgt vocab size: 10036.
[2022-06-10 09:44:29,624 INFO]  * src vocab size: 9808.
[2022-06-10 09:44:29,742 INFO] Building & saving validation data...
[2022-06-10 09:44:29,798 INFO] Building shard 0.
[2022-06-10 09:44:29,811 INFO]  * saving 0th valid data shard to translate/data.valid.0.pt.


### train

In [ ]:
!rm -r translate

In [ ]:
!python OpenNMT-py/train.py -data translate/data -save_model translate/model -param_init 0 -param_init_glorot -label_smoothing 0.1 -enc_layers 2 --dec_layers 2 -rnn_size 512 -word_vec_size 512 -batch_size 4096 -batch_type tokens -optim adam -adam_beta2 0.998 -learning_rate 0.001 -normalization tokens -encoder_type transformer -decoder_type transformer -train_steps 20000 -save_checkpoint_steps 2000  -valid_steps 1000 -world_size 1 -gpu_rank 0

[2022-06-10 09:46:22,073 INFO]  * src vocab size = 9808
[2022-06-10 09:46:22,073 INFO]  * tgt vocab size = 10036
[2022-06-10 09:46:22,073 INFO] Building model...
[2022-06-10 09:46:26,144 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(9808, 512, padding_idx=1)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=512, out_features=512, bias=True)
          (linear_values): Linear(in_features=512, out_features=512, bias=True)
          (linear_query): Linear(in_features=512, out_features=512, bias=True)
          (softmax): Softmax(dim=-1)
          (dropout): Dropout(p=0.1, inplace=False)
          (final_linear): Linear(in_features=512, out_features=512, bias=True)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): 

### test

In [ ]:
!rm -r pred

In [ ]:
!mkdir pred
!mkdir pred/test
!mkdir pred/validation

In [ ]:
bleus = []
for i in tqdm(range(2000,20001,2000)) :
  c1 = !python OpenNMT-py/translate.py -model translate/model_step_{i}.pt -src data/valid.en -output pred/validation/pred_valid_{i}.txt -replace_unk
  c2 = !sed -i "s/@@ //g"  pred/validation/pred_valid_{i}.txt
  c3 = !perl OpenNMT-py/tools/multi-bleu.perl data/valid.fa < pred/validation/pred_valid_{i}.txt
  bleus.append(c3)

100%|██████████| 10/10 [07:59<00:00, 47.93s/it]


In [ ]:
bleus

[['Use of uninitialized value in division (/) at OpenNMT-py/tools/multi-bleu.perl line 139, <STDIN> line 400.',
  'BLEU = 0.00, 7.3/0.7/0.1/0.0 (BP=1.000, ratio=1.471, hyp_len=9300, ref_len=6322)'],
 ['BLEU = 0.57, 9.8/0.9/0.2/0.1 (BP=1.000, ratio=1.307, hyp_len=8266, ref_len=6322)'],
 ['BLEU = 0.44, 13.5/1.3/0.2/0.0 (BP=0.873, ratio=0.881, hyp_len=5567, ref_len=6322)'],
 ['BLEU = 0.94, 13.1/1.3/0.3/0.2 (BP=1.000, ratio=1.090, hyp_len=6888, ref_len=6322)'],
 ['BLEU = 1.00, 17.3/2.0/0.4/0.1 (BP=0.860, ratio=0.869, hyp_len=5496, ref_len=6322)'],
 ['BLEU = 0.66, 20.8/2.5/0.6/0.1 (BP=0.547, ratio=0.624, hyp_len=3942, ref_len=6322)'],
 ['BLEU = 0.81, 16.4/2.1/0.3/0.1 (BP=0.889, ratio=0.895, hyp_len=5656, ref_len=6322)'],
 ['BLEU = 0.57, 21.0/2.8/0.4/0.0 (BP=0.629, ratio=0.683, hyp_len=4318, ref_len=6322)'],
 ['BLEU = 0.75, 21.1/2.8/0.6/0.1 (BP=0.542, ratio=0.620, hyp_len=3919, ref_len=6322)'],
 ['BLEU = 0.79, 23.0/3.0/0.7/0.1 (BP=0.520, ratio=0.605, hyp_len=3822, ref_len=6322)']]

In [ ]:
!python OpenNMT-py/translate.py -model translate/model_step_10000.pt -src data/test.en -output pred/test/pred_test.txt -replace_unk -verbose

[2022-06-10 10:57:08,197 INFO] Translating shard 0.
/content/OpenNMT-py/onmt/translate/beam_search.py:209: UserWarning: An output with one or more elements was resized since it had shape [150], which does not match the required output shape [30, 5].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:24.)
  torch.mul(self.topk_scores, length_penalty, out=self.topk_log_probs)
/content/OpenNMT-py/onmt/translate/beam_search.py:212: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use 

In [ ]:
!sed -i "s/@@ //g"  pred/test/pred_test.txt

In [ ]:
!perl OpenNMT-py/tools/multi-bleu.perl data/test.fa < pred/test/pred_test.txt

BLEU = 0.61, 14.7/1.6/0.2/0.0 (BP=0.976, ratio=0.977, hyp_len=11120, ref_len=11385)
